# Label Transfer
In this Notebook a NN model is used to classify unannoted cells and reclassify cells whose transcriptomic profile changed.

In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/bin/python
3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
sys.version_info(major=3, minor=9, micro=18, releaselevel='final', serial=0)


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))#embed HTML within IPython notebook and make containers 90% of screen
import scanpy as sc
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import numpy as np
import gdown
import copy as cp
import seaborn as sns
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

%matplotlib inline
#scvi.settings.seed = 0
sc.settings.verbosity = 0             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, facecolor='white') #figure resolution and background color


/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


In [3]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)
#To make reproducible plots
np.random.seed(41)

In [4]:
#set data path
os.chdir('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/')
sc.settings.figdir = '/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scarches/'
np.random.seed(41)

In [ ]:
path='/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scvi/models/'
files=[f'{path}scvi_model_integration_l50/adata_labels.h5ad',f'{path}scvi_model_final/adata_labekl.h5ad']
names=['labels','final']
object_names_merged=list()
numberobj = len(files)

for i in range(numberobj):
    object_names_merged.append("adata"+"_"+names[i])
    locals()[object_names_merged[i]] = sc.read_h5ad(files[i]) 
    print(object_names_merged[i],' is loaded')

In [13]:
adata_final

AnnData object with n_obs × n_vars = 396794 × 2000
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', '_scvi_batch', '_scvi_labels', 'leiden', 'cell_type_low_res', 'cell_type_high_res', 'harmonized_cell_type', 'harmonized_cell_

# K-nn prediction on lowest resolution and label transfer
Need all annotated cells as training data and the whole dataset to be reclassified. (This does not have to be done this way)

In [23]:
adata_final=sc.read_h5ad('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scvi/models/scvi_model_final/adata_known_harmonized.h5ad')

In [18]:
adata_combined=sc.read_h5ad('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scvi/models/scvi_model_final/adata_latent_all.h5ad')

Split data between labelled for training and unknown that needs to be predicted.

In [51]:
for i in ['harmonized_cell_type_unfiltered_scvi',
       'harmonized_cell_type_uncert_scvi',
       'harmonized_cell_type_filtered_02_scvi',
       'harmonized_cell_type_unfiltered_harmony',
       'harmonized_cell_type_uncert_harmony',
       'harmonized_cell_type_filtered_02_harmony',
       'harmonized_cell_type_unfiltered_scanorama',
       'harmonized_cell_type_uncert_scanorama',
       'harmonized_cell_type_filtered_02_scanorama'
         ]:
    del adata_combined.obs[i]
    del adata_final.obs[i]

In [24]:

#adata_ref=adata_final.copy()
adata_query=adata_combined[~adata_combined.obs_names.isin(adata_final.obs_names)]

In [25]:
adata_query

View of AnnData object with n_obs × n_vars = 522220 × 90
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'sample', 'atlas', 'species', 'gender', 'age', 'instrument', 'technology', 'donor', 'cell_type_lowest_res', 'cell_type_low_res', 'cell_type_med_res', 'cell_type_high_res', 'harmonized_cell_type_lowest_res'

In [1]:
a = set(adata_ref.obs["cell_type_lowest_res"].tolist())
cell_type_low_res_batch=adata_ref.obs["cell_type_lowest_res"].tolist()
for i in a:
    c=0
    for j in cell_type_low_res_batch:
        if j == i:
            c+=1
    print(f'{i}: {c}')
    

NameError: name 'adata_ref' is not defined

In [24]:
set(adata_ref.obs['harmonized_cell_type_lowest_res'].tolist())

{'Astrocytes-smajic',
 'ENDO-welch',
 'EPEN-siletti',
 'Endothelial cells-smajic',
 'Endothelial-agarwal',
 'MISC-siletti',
 'Microglia-smajic',
 'NEUR-siletti',
 'OLIGO-siletti',
 'OPC-siletti',
 'Pericytes-smajic'}

In [13]:
knn_transformer_scvi = sca.utils.knn.weighted_knn_trainer(
    train_adata=adata_ref,
    train_adata_emb="X_scVI",  # location of our joint embedding
    n_neighbors=30,
)
labels_scvi, uncert_scvi = sca.utils.knn.weighted_knn_transfer(
    query_adata=adata_query,
    query_adata_emb="X_scVI",  # location of our embedding, query_adata.X in this case
    label_keys="harmonized_cell_type_lowest_res",  # (start of) obs column name(s) for which to transfer labels
    knn_model=knn_transformer_scvi,
    ref_adata_obs=adata_ref.obs,
)
uncertainty_threshold = 0.2
labels_scvi.rename(
    columns={
        "harmonized_cell_type_lowest_res": "harmonized_lowest_unfiltered_scvi"
    },
    inplace=True,
)
uncert_scvi.rename(
    columns={
        "harmonized_cell_type_lowest_res": "harmonized_lowest_uncert_scvi"
    },
    inplace=True,
)
adata_combined.obs = adata_combined.obs.join(labels_scvi)
adata_combined.obs = adata_combined.obs.join(uncert_scvi)

adata_combined.obs['harmonized_lowest_filtered_02_scvi'] = adata_combined.obs[
    "harmonized_lowest_unfiltered_scvi"
].mask(
    adata_combined.obs["harmonized_lowest_uncert_scvi"] > uncertainty_threshold,
    "Unknown",
)
print(
    f"Percentage of unknown for scvi, with uncertainty_threshold={uncertainty_threshold}:"
)
print(
    f"Filtered: {np.round(sum(adata_combined.obs['harmonized_lowest_filtered_02_scvi'] =='Unknown')/adata_query.n_obs*100,2)}% of previously unknown"
)


Weighted KNN with n_neighbors = 30 ... finished!
Percentage of unknown for scvi, with uncertainty_threshold=0.2:
Filtered: 2.77% of previously unknown


In [15]:
adata_combined.obs['harmonized_lowest_filtered_02_scvi']

N3_CTGTGCTTCAGAAATG                      Unknown
N3_GCTCCTACAGATCCAT                      Unknown
N3_TCAGCAATCGGTCCGA                      Unknown
N3_TGACTAGTCACAGGCC                      Unknown
N3_TACGGATTCTGTACGA                      Unknown
                                  ...           
TGGGAGATCACGGTCG-1-1    Endothelial cells-smajic
TGTTGGACATCTTCGC-1                 OLIGO-siletti
TTACGTTGTCAACCTA-1              Microglia-smajic
TGTACAGAGGTGCGAT-1-1                NEUR-siletti
TTGCGTCGTCAAGCCC-1                 OLIGO-siletti
Name: harmonized_lowest_filtered_02_scvi, Length: 522220, dtype: object

In [20]:
a = set(adata_combined.obs['harmonized_lowest_filtered_02_scvi'].tolist())
cell_type_low_res_batch=adata_combined.obs['harmonized_lowest_filtered_02_scvi'].tolist()
for i in a:
    c=0
    for j in cell_type_low_res_batch:
        if j == i:
            c+=1
    print(f'{i}: {c}')
    

OPC-siletti: 30575
EPEN-siletti: 563
Astrocytes-smajic: 39179
NEUR-siletti: 157244
Unknown: 14457
OLIGO-siletti: 230390
Endothelial cells-smajic: 1400
Pericytes-smajic: 468
Microglia-smajic: 41173
ENDO-welch: 4394
MISC-siletti: 2377


In [26]:
adata_combined.obs=adata_combined.obs.astype(str) #uncert are not str, doesnt allow me to save h5ad

In [21]:
adata_filtered=adata_combined[adata_combined.obs['harmonized_lowest_filtered_02_scvi']!='Unknown']

In [42]:
adata_filtered

View of AnnData object with n_obs × n_vars = 510294 × 2000
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', '_scvi_batch', '_scvi_labels', 'leiden', 'cell_type_low_res', 'cell_type_high_res', 'harmonized_cell_type', 'harmoniz

In [46]:
set(adata_combined.obs['original_cellclass_lvl1'].tolist())

{'',
 'Astrocyte',
 'Endothelial',
 'Microglia',
 'Neuron',
 'None',
 'OPC',
 'Oligodendrocyte',
 'PVM',
 'Perivascular',
 'T-Cell'}

In [47]:
adata_combined[adata_combined.obs['original_cellclass_lvl1']=='Neuron']

View of AnnData object with n_obs × n_vars = 1483 × 2000
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', '_scvi_batch', '_scvi_labels', 'leiden', 'cell_type_low_res', 'cell_type_high_res', 'harmonized_cell_type', 'harmonized

In [24]:
adata_combined.write('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scvi/models/scvi_model_final/adata_predicted_lowest.h5ad')